In [1]:
import pandas as pd
import leafmap
import geopandas as gpd
import folium
import seaborn as sns
import sys

In [2]:
data = pd.read_csv('../../data/EnergyAdinelsa/consumo energia 202407.csv', sep=';')

# Load the shapefile using geopandas
in_shp = "../../data/DATOS_GEOGRAFICOS_PERU/03_DISTRITOS/LIMITE_DISTRITAL.shp"
gdf = gpd.read_file(in_shp)


In [3]:
data.head(3)

,FECHA_CORTE,PERIODO_FACTURACION,UUID,UBIGEO,DEPARTAMENTO,PROVINCIA,DISTRITO,LOCALIDAD,TARIFA,FECHA_INICIO,CONSUMO_KW,POTENCIA_CONTRATADA,MONTO_SOLES,FASE,CODIGO_CIU
0,20240809,202407,C202201170199124880220119,151011,LIMA,YAUYOS,COLONIA,COLONIA,BT5B,20220117,199.0,1.0,237.3,MONOFASICO,T9700
1,20240809,202407,C202203060199127920220314,151001,LIMA,YAUYOS,YAUYOS,YAUYOS,BT5B,20220306,12.0,3.0,38.8,MONOFASICO,T9700
2,20240809,202407,C202204160199130660220412,151003,LIMA,YAUYOS,AYAUCA,CONCUBAY,BT5B,20220416,38.0,3.0,35.9,MONOFASICO,T9700


In [4]:
data.drop(axis=1, columns=['FECHA_CORTE', 'FECHA_INICIO'], inplace=True)

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80586 entries, 0 to 80585
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   PERIODO_FACTURACION  80586 non-null  int64  
 1   UUID                 80586 non-null  object 
 2   UBIGEO               80586 non-null  int64  
 3   DEPARTAMENTO         80586 non-null  object 
 4   PROVINCIA            80586 non-null  object 
 5   DISTRITO             80586 non-null  object 
 6   LOCALIDAD            80586 non-null  object 
 7   TARIFA               80586 non-null  object 
 8   CONSUMO_KW           80586 non-null  float64
 9   POTENCIA_CONTRATADA  80586 non-null  float64
 10  MONTO_SOLES          80586 non-null  float64
 11  FASE                 80586 non-null  object 
 12  CODIGO_CIU           80586 non-null  object 
dtypes: float64(3), int64(2), object(8)
memory usage: 8.0+ MB


In [6]:
data.columns

Index(['PERIODO_FACTURACION', 'UUID', 'UBIGEO', 'DEPARTAMENTO', 'PROVINCIA',
       'DISTRITO', 'LOCALIDAD', 'TARIFA', 'CONSUMO_KW', 'POTENCIA_CONTRATADA',
       'MONTO_SOLES', 'FASE', 'CODIGO_CIU'],
      dtype='object')

In [23]:
df = data.groupby(['DEPARTAMENTO', 'PROVINCIA', 'DISTRITO', 'POTENCIA_CONTRATADA'], as_index=False).agg({'CONSUMO_KW': 'mean', 'MONTO_SOLES': 'mean'})

In [18]:
gdf.columns

Index(['OBJECTID', 'UBIGEO', 'CCDD', 'CCPP', 'CCDI', 'DEPARTAMENTO',
       'PROVINCIA', 'DISTRITO', 'geometry'],
      dtype='object')

In [19]:
gdf.rename(columns={'DEPARTAMEN':'DEPARTAMENTO'}, inplace=True)

In [16]:
gdf.columns

Index(['OBJECTID', 'UBIGEO', 'CCDD', 'CCPP', 'CCDI', 'DEPARTAMENTO',
       'PROVINCIA', 'DISTRITO', 'geometry'],
      dtype='object')

In [22]:
gdf.drop(columns=['OBJECTID', 'UBIGEO', 'CCDD', 'CCPP', 'CCDI'], inplace=True)

In [24]:
df = gdf.merge(df, on=['DEPARTAMENTO', 'PROVINCIA', 'DISTRITO'], how='inner')

In [27]:
leafmap.search_xyz_services(keyword="esri")

['xyz.Esri.WorldStreetMap',
 'xyz.Esri.WorldTopoMap',
 'xyz.Esri.WorldImagery',
 'xyz.Esri.WorldTerrain',
 'xyz.Esri.WorldShadedRelief',
 'xyz.Esri.WorldPhysical',
 'xyz.Esri.OceanBasemap',
 'xyz.Esri.NatGeoWorldMap',
 'xyz.Esri.WorldGrayCanvas',
 'xyz.Esri.ArcticImagery',
 'xyz.Esri.ArcticOceanBase',
 'xyz.Esri.ArcticOceanReference',
 'xyz.Esri.AntarcticImagery',
 'xyz.Esri.AntarcticBasemap']

In [26]:
df.to_file('../data/data.shp', driver='ESRI Shapefile')
# Convert and save it as GeoJSON
#geojson_output_path = "data.geojson"
#df.to_file(geojson_output_path, driver="GeoJSON")

In [30]:
gpd.read_file('../data/data.shp').drop(columns=['OBJECTID', 'UBIGEO', 'CCDD', 'CCPP', 'CCDI']).to_file('../data/data.shp', driver='ESRI Shapefile')

In [10]:
# Create a Leafmap map centered at specified coordinates
m = leafmap.Map(center=[-12.046374, -77.042793], zoom=5)

# Add the choropleth layer
m.add_data(
    data=df,
    column='POTENCIA_CONTRATADA',
    scheme="NaturalBreaks", 
    cmap="Blues", 
    add_legend=True,
    legend_kwds={"fmt": "{:.0f}"},
    legend_title="Population",
    )

# Add layer control to the map
m

Map(center=[-12.046374, -77.042793], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title…